# Update Graph Data

Run this to update schema and the related information for the graph.

## Workspace Setup

In [1]:
# Load credentials and other environment variables
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# Graph database credentials
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
pwd = os.getenv("NEO4J_PWD")

# Relevant files
schema_prompt_file = os.getenv("SCHEMA_PROMPT")
schema_full_file = os.getenv("SCHEMA_FULL")
schema_simplified_file = os.getenv("SCHEMA_SIMPLIFIED")
nodes_file = os.getenv("NODES_LIST")
node_props_file = os.getenv("NODES_PROPERTIES")
rels_props_file = os.getenv("RELATIONSHIPS_PROPERTIES")
rels_file = os.getenv("RELATIONSHIPS_LIST")
instances_nodes_file = os.getenv("INSTANCES_NODES")
instances_relationships_file = os.getenv("INSTANCES_RELATIONSHIPS")

In [2]:
# Set the working directory and import the local modules
import sys
from pathlib import Path
parent_dir = Path.cwd() #.parent
sys.path.append(str(parent_dir))

from helpers.utilities import *
from helpers.graph_utils import *
from helpers.neo4j_schema import Neo4jSchema

In [4]:
# Instantiate the schema utilities module 
sutils = Neo4jSchema(uri, user, pwd)

## Update & Save Graph Information

In [5]:
# Simplified schema as a text to be used for prompt
schema_simplified = sutils.build_string_schema_simplified()
write_json(schema_simplified, schema_prompt_file)

# Create the full graph schema and save it to a file
sutils.save_schema_full(schema_full_file)
#schema_full = read_json(schema_full_file)

# Create the simplified graph schema and save it to a file
sutils.save_schema_simplified(schema_simplified_file)
#schema_simplified = read_json(schema_simplified_file)

# Save nodes labels to a file 
sutils.get_node_names(nodes_file)
#nodes = read_json(nodes_file)

# Save the nodes with their properties and data types
sutils.node_properties_with_types(node_props_file)
#node_props_types = read_json(node_props_file)

# Save the relationship properties
sutils.relationship_properties_with_types(rels_props_file)
#rels_props_types = read_json(rels_props_file)

# Save the relationships
sutils.relationships(rels_file)
#rels = read_json(rels_file)

## Update & Save Instances Data

### Node Instances

In [5]:
# Read the nodes list
nodes = read_json(nodes_file)

# Extract n=10 instances for each node label in the graph
nodes_instances = sutils.extract_data_values(nodes, 10)

# Serialize the instances data by parsing date, datetime properties
new_nodes_instances = serialize_node_data(nodes_instances)

# Save the parsed instances to a file
write_json(new_nodes_instances, instances_nodes_file)

## Relationship Instances

In [5]:
# Read the relationships list
rels = read_json(rels_file)

# Extract relationships as triples: [source, relationship, target]
relationships_triples = extract_relationships_triples(rels)

# Extract parsed relationships instances - one sublist per relationship triple
rels_instances = sutils.extract_multiple_relationships_instances(relationships_triples, 10)

# Serialize the instances data by parsing date, datetime properties
new_rels_instances = serialize_rels_data(rels_instances)

# Add the node labels to the relationships information
parsed_rels_instances = combine_rels_with_labels(relationships_triples,new_rels_instances)

# Save the parsed instances to a file
write_json(parsed_rels_instances, instances_relationships_file)